In [138]:
import pandas as pd

**Задание 1**

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [139]:
df = pd.read_csv('ml-latest-small/ratings.csv')
df = df.groupby('userId').agg({'rating':'count', 'timestamp':['max','min']})
df.head()

rating   timestamp            
        count         max         min
userId                               
1          20  1260759205  1260759108
2          76   835356246   835355395
3          51  1298932787  1298861589
4         204   949982274   949778714
5         100  1163375145  1163373044

In [140]:
df = df[df['rating']['count'] > 100]
df['lifetime'] = df['timestamp']['max'] - df['timestamp']['min']
df.head()

rating   timestamp               lifetime
        count         max         min           
userId                                          
4         204   949982274   949778714     203560
8         116  1154474527  1154389340      85187
15       1700  1469330735   997937239  471393496
17        363  1127476640  1127468587       8053
19        423   855195373   855190091       5282

In [141]:
# df.lifetime.mean() - среднее время жизни пользователя в формате unix

pd.Timedelta(df.lifetime.mean(), unit='s')

Timedelta('463 days 21:28:27.449612')

**Задание  2**

Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
- таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
- аналогичную таблицу по типам выручки с указанием адреса клиента

Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

Дана статистика услуг перевозок клиентов компании по типам:
- rzd - железнодорожные перевозки
- auto - автомобильные перевозки
- air - воздушные перевозки
- client_base - адреса клиентов

In [142]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [143]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [144]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [145]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base.set_index('client_id', inplace=True)
client_base

,address
client_id,
111,Комсомольская 4
112,Энтузиастов 8а
113,Левобережная 1а
114,Мира 14
115,ЗЖБИиДК 1
116,Строителей 18
117,Панфиловская 33
118,Мастеркова 4


In [146]:
# Таблица с тремя типами выручки для каждого client_id без указания адреса клиента

client_base.join(auto.set_index('client_id')).join(rzd.set_index('client_id'))\
        .join(air.set_index('client_id'))[['auto_revenue', 'rzd_revenue','air_revenue']]

,auto_revenue,rzd_revenue,air_revenue
client_id,,,
111,NaN,1093.0,NaN
112,NaN,2810.0,NaN
113,57483.0,10283.0,NaN
114,83.0,5774.0,NaN
115,912.0,981.0,81.0
116,4834.0,NaN,4.0
117,98.0,NaN,13.0
118,NaN,NaN,173.0


In [147]:
# Таблица по типам выручки с указанием адреса клиента

client_base.join(auto.set_index('client_id')).join(rzd.set_index('client_id'))\
        .join(air.set_index('client_id'))

,address,auto_revenue,rzd_revenue,air_revenue
client_id,,,,
111,Комсомольская 4,NaN,1093.0,NaN
112,Энтузиастов 8а,NaN,2810.0,NaN
113,Левобережная 1а,57483.0,10283.0,NaN
114,Мира 14,83.0,5774.0,NaN
115,ЗЖБИиДК 1,912.0,981.0,81.0
116,Строителей 18,4834.0,NaN,4.0
117,Панфиловская 33,98.0,NaN,13.0
118,Мастеркова 4,NaN,NaN,173.0


**Задание 3**

В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:
- У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
- Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?
- Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

-----

Во-первых стоит понять логику сбора данных о координатах. Если, например, координаты определялись при каждом входе в приложение, то они должны быть напрямую связаны с таблицей визитов по времени визита. Если координат меньше, чем визитов, то соотносим координаты с ближайшим предшествующим визитом или ближайше последующей покупкой. Можно, наверное, еще наложить ограничение на промежуток времени между зафиксированнм визитом/покупкой и когда были зафиксированы координаты, не более часа или суток.

Из координат можно получить регион местонахождения пользователя, часовой пояс. Определить подозрпительную активность, если координаты сильно отличаются от обычных для пользователя.